In [1]:
from skimage import io, measure
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import tifffile
import imageio
from PIL import Image
import glob
import multiprocessing as mp
import re
import os 
Image.MAX_IMAGE_PIXELS = None

In [10]:
base_path = 'Seurat_Analysis/Bone_Marrow/'
sample_names = ['H10', 'H14', 'H26', 'H27', 'H32', 'H33', 'H35', 'H36', 'H37', 'H38', 'H39', 'H41']
#sample_names = ['H26']

In [3]:
#Convert binary masks of fibro MSC to segmentation TIFF files 
for sample_name in sample_names:
    # Load the binary mask PNG file for the current sample
    file_path = f"{base_path}Fibro-MSC_Masks/Fibro-MSC_{sample_name}_Scan1.qptiff - resolution #1-labels.png"
    binary_mask = io.imread(file_path, as_gray=True)
    
    # Convert the binary mask to a numpy array (ensure it's binary)
    binary_mask = np.array(binary_mask, dtype=bool)
    
    # Label connected components in the binary mask
    labeled_cells = measure.label(binary_mask)
    unique_labels = np.unique(labeled_cells)
    seg_mask = np.zeros_like(labeled_cells, dtype=np.uint16)
    
    for idx, label in enumerate(unique_labels[1:]):  # Skip 0 (background label)
        seg_mask[labeled_cells == label] = idx + 1  # Assign unique values to each cell
    
    print(f"Total cells in {sample_name}: {len(unique_labels) - 1}")  # Subtract background label
    
    # Save the labeled cells as a TIFF file
    output_path = f"{base_path}Fibro-MSC_Masks/{sample_name}_Fibro_MSC_Scan1_Segmentation.tiff"
    io.imsave(output_path, seg_mask.astype(np.uint8), plugin='tifffile', check_contrast=False)

Total cells in H10: 9
Total cells in H14: 3
Total cells in H26: 3
Total cells in H27: 3
Total cells in H32: 1
Total cells in H33: 2
Total cells in H35: 5
Total cells in H36: 2
Total cells in H37: 1
Total cells in H38: 1
Total cells in H39: 9
Total cells in H41: 25


In [8]:
#Convert binary masks of osteo MSC to segmentation TIFF files 
for sample_name in sample_names:
    # Load the binary mask PNG file for the current sample
    file_path = f"{base_path}Osteo-MSC_Masks/Osteo-MSC_{sample_name}_Scan1.qptiff - resolution #1-labels.png"
    binary_mask = io.imread(file_path, as_gray=True)
    
    # Convert the binary mask to a numpy array (ensure it's binary)
    binary_mask = np.array(binary_mask, dtype=bool)
    
    # Label connected components in the binary mask
    labeled_cells = measure.label(binary_mask)
    unique_labels = np.unique(labeled_cells)
    seg_mask = np.zeros_like(labeled_cells, dtype=np.uint16)
    
    for idx, label in enumerate(unique_labels[1:]):  # Skip 0 (background label)
        seg_mask[labeled_cells == label] = idx + 1  # Assign unique values to each cell
    
    print(f"Total cells in {sample_name}: {len(unique_labels) - 1}")  # Subtract background label
    
    # Save the labeled cells as a TIFF file
    output_path = f"{base_path}Osteo-MSC_Masks/{sample_name}_Osteo_MSC_Scan1_Segmentation.tiff"
    io.imsave(output_path, seg_mask.astype(np.uint8), plugin='tifffile', check_contrast=False)

Total cells in H26: 16


Quantify expression for each of the manually annotated masks 

In [5]:
def sumValueByMask(mask, expr, outFile = ""):
    df = pd.DataFrame({
        'values':  np.concatenate(expr),
        'indexes': np.concatenate(mask)
    })
    out = df.pivot_table(values='values', index='indexes', aggfunc=sum)
    if(outFile != ""):
        out.to_csv(outFile)
    return out

In [ ]:
#Quantify expression for Fibro-MSC 
for sample_name in sample_names:
    print('Now Processing:')
    print(sample_name)
    quant_path = os.path.join(base_path, f'Fibro-MSC_Masks/{sample_name}_Quant')
    if not os.path.exists(quant_path):
        os.makedirs(quant_path)
    
    inOmeTiff = os.path.join(base_path, f'CODEX_Images/{sample_name}_Scan1.qptiff')
    inMaskTiff = os.path.join(base_path, f'Fibro-MSC_Masks/{sample_name}_Fibro_MSC_Scan1_Segmentation.tiff')
    
    im = tifffile.imread(inOmeTiff)
    print('Image Size')
    print(im.shape)
    
    mask = tifffile.imread(inMaskTiff)
    print('Mask Size:')
    print(mask.shape)
    
    df = pd.DataFrame({
      'values': np.concatenate(np.full(mask.shape, 1)),
      'indexes': np.concatenate(mask)
    })

    # Example paths for saving based on the sample_name
    size = sumValueByMask(mask, np.full(mask.shape, 1), os.path.join(quant_path, 'cell_size.csv'))

    coordX = sumValueByMask(
        mask, 
        np.tile(
            np.array(range(0, mask.shape[1])), 
            (mask.shape[0], 1)),
        os.path.join(quant_path, 'cell_x_coord.csv'))

    coordY = sumValueByMask(
        mask, 
        np.transpose(np.tile(
            np.array(range(0, mask.shape[0])), 
            (mask.shape[1], 1))),
        os.path.join(quant_path, 'cell_y_coord.csv'))   

    outFiles = [os.path.join(quant_path, f'channel_C{ii:02d}.csv') for ii in range(im.shape[0])]

    for ii in range(len(outFiles)):
        print(ii)
        sumValueByMask(mask, im[ii,:,:], outFiles[ii])

    print('Creating master csv')
    csvDir = quant_path
    csvFileArray = sorted(glob.glob(csvDir + "/*.csv"))
    print("Output Directory is " + csvDir)
    df_list = []
    for filename in csvFileArray:
        df = pd.read_csv(filename, header=0)
        df = df.rename(columns={"values": str(filename)})  # changes "values" to the filename
        df_list.append(df)
    col_names = [str(i) for i in csvFileArray]
    dfs = [df.set_index('indexes') for df in df_list]
    frame = pd.concat(dfs, axis=1)
    print(np.shape(frame))  # to confirm that the data-frame has written the correct number of columns
    frame.to_csv(os.path.join(csvDir, 'combined_markers.csv'), index=True)


In [ ]:
#Quantify expression for Osteo-MSC 
for sample_name in sample_names:
    print('Now Processing:')
    print(sample_name)
    quant_path = os.path.join(base_path, f'Osteo-MSC_Masks/{sample_name}_Quant')
    if not os.path.exists(quant_path):
        os.makedirs(quant_path)
    
    inOmeTiff = os.path.join(base_path, f'CODEX_Images/{sample_name}_Scan1.qptiff')
    inMaskTiff = os.path.join(base_path, f'Osteo-MSC_Masks/{sample_name}_Osteo_MSC_Scan1_Segmentation.tiff')
    
    im = tifffile.imread(inOmeTiff)
    print('Image Size')
    print(im.shape)
    
    mask = tifffile.imread(inMaskTiff)
    print('Mask Size:')
    print(mask.shape)
    
    df = pd.DataFrame({
      'values': np.concatenate(np.full(mask.shape, 1)),
      'indexes': np.concatenate(mask)
    })

    # Example paths for saving based on the sample_name
    size = sumValueByMask(mask, np.full(mask.shape, 1), os.path.join(quant_path, 'cell_size.csv'))

    coordX = sumValueByMask(
        mask, 
        np.tile(
            np.array(range(0, mask.shape[1])), 
            (mask.shape[0], 1)),
        os.path.join(quant_path, 'cell_x_coord.csv'))

    coordY = sumValueByMask(
        mask, 
        np.transpose(np.tile(
            np.array(range(0, mask.shape[0])), 
            (mask.shape[1], 1))),
        os.path.join(quant_path, 'cell_y_coord.csv'))   

    outFiles = [os.path.join(quant_path, f'channel_C{ii:02d}.csv') for ii in range(im.shape[0])]

    for ii in range(len(outFiles)):
        print(ii)
        sumValueByMask(mask, im[ii,:,:], outFiles[ii])

    print('Creating master csv')
    csvDir = quant_path
    csvFileArray = sorted(glob.glob(csvDir + "/*.csv"))
    print("Output Directory is " + csvDir)
    df_list = []
    for filename in csvFileArray:
        df = pd.read_csv(filename, header=0)
        df = df.rename(columns={"values": str(filename)})  # changes "values" to the filename
        df_list.append(df)
    col_names = [str(i) for i in csvFileArray]
    dfs = [df.set_index('indexes') for df in df_list]
    frame = pd.concat(dfs, axis=1)
    print(np.shape(frame))  # to confirm that the data-frame has written the correct number of columns
    frame.to_csv(os.path.join(csvDir, 'combined_markers.csv'), index=True)
